<a href="https://colab.research.google.com/github/snklp/pytorch/blob/pt/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim

**Model**

In [ ]:
class CNN(nn.Module):
  def __init__(self, ip_channels=1, op_features=10):
    super().__init__()
    self.conv1 = nn.Conv2d(ip_channels, 8, 3, (1,1), (1,1) )  # ip_channnels, op_channels, kernel_size, stride, padding
    self.conv2 = nn.Conv2d(8, 16, 3, (1,1), (1,1))
    self.maxpool = nn.MaxPool2d((2,2), (2,2))   # kernel_size, stride
    self.fc1 = nn.Linear(16*7*7, op_features)
    
  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.maxpool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.maxpool(x)
    x = x.view(x.shape[0], -1)
    x = self.fc1(x)
    return x



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Hyperparameters**

In [ ]:
ip_channels = 1
op_features = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

# in case we wanna load the checkpoints
load_model = False

**Loading Data**

In [ ]:
train_dataset = datasets.MNIST(root='/content/drive/MyDrive/Datasets', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='/content/drive/MyDrive/Datasets', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model = CNN().to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

if load_model:
  checkpoint = torch.load('mycheckpoint.pth.tar')      # importing checkpoints from the file
  model.load_state_dict(checkpoint['state_dict'])      # loading model checkpoint
  optimizer.load_state_dict(checkpoint['optimizer'])   # loading optimizer checkpoints

**Training**

In [ ]:
for epoch in range(num_epochs):
  checkpoint = {'state_dict':model.state_dict(), 'optimizer':optimizer.state_dict()}  # stores checkpoint in the dic
  torch.save(checkpoint, 'mycheckpoint.pth.tar')                                      # saves the stored checkpoints in dic to the file 
  
  for data in train_loader:
    X, y = data
    X = X.to(device)
    y = y.to(device)
    #print(X.shape)
    #break
    y_ = model(X)
    loss = loss_function(y_, y)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()


**Testing**

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for data in test_loader:
    X, y = data
    X = X.to(device)
    y = y.to(device)
    y_ = model(X)

    for idx, op_class in enumerate(y_):
      if torch.argmax(op_class) == y[idx]:correct += 1
      total += 1

**Accuracy**

In [ ]:
correct/total

0.9664